In [11]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import os
import s3fs
import matplotlib.pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.regularizers import l2


import xgboost

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import mean_squared_error, accuracy_score 
from xgboost import XGBClassifier

from modules.utils import pre_process_data, encoded_categorical_features

In [18]:
X_test = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_test_Hi5.csv", sep=",")
X_test["piezo_measurement_date"] = pd.to_datetime(X_test["piezo_measurement_date"], errors="coerce")
X_test["piezo_groundwater_level_category"] = 0

x_test_without_nan, y_train, y_origginal = pre_process_data(X_test)

/var/folders/41/lv7xy5r91x7ckkgx5zqmgpxh0000gn/T/ipykernel_10888/1734340539.py:1: DtypeWarning: Columns (1,5,108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_test_Hi5.csv", sep=",")
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/pytho

['prelev_volume_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_volume_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_volume_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2', 'prelev_other_volume_sum']


/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  retur

In [2]:
x_train = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_train_Hi5.csv", sep=",")

/var/folders/41/lv7xy5r91x7ckkgx5zqmgpxh0000gn/T/ipykernel_10888/3534149467.py:1: DtypeWarning: Columns (108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  x_train = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_train_Hi5.csv", sep=",")


In [12]:
print(len(x_train.columns))

136


In [4]:

# Convert to datetime
x_train["piezo_measurement_date"] = pd.to_datetime(x_train["piezo_measurement_date"], errors="coerce")

In [16]:
print(len(x_train.columns))

136


In [31]:
x_train = x_train[x_train["piezo_measurement_date"].dt.month.isin([6, 7, 8, 9])]

In [32]:
print(len(x_train.columns))

136


In [ ]:
x_train["piezo_measurement_date"].value_counts()

In [ ]:
x_train.info()

In [33]:
x_trained_without_nan, y_train, y_original = pre_process_data(x_train)

/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  retur

['prelev_volume_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_volume_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_volume_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2', 'prelev_other_volume_sum']


/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  retur

In [34]:
print(len(x_trained_without_nan.columns))

87


In [35]:
print(len(x_test_without_nan.columns))

86


In [22]:
df1_columns = set(x_test_without_nan.columns)
df2_columns = set(x_trained_without_nan.columns)

# Find common columns
common_columns = df1_columns.intersection(df2_columns)
print(f"Common columns: {common_columns}")

# Find columns unique to df1
unique_to_df1 = df1_columns - df2_columns
print(f"Columns in df1 but not in df2: {unique_to_df1}")

# Find columns unique to df2
unique_to_df2 = df2_columns - df1_columns
print(f"Columns in df2 but not in df1: {unique_to_df2}")

# Compare the data of the common columns


Common columns: {'piezo_station_department_code', 'meteo__pressure_saturation_avg', 'meteo_if_frost', 'meteo_evapotranspiration_Monteith', 'meteo_time_wind_avg', 'meteo_temperature_avg_threshold', 'meteo_temperature_max', 'meteo_if_smoke', 'meteo_evapotranspiration_grid', 'meteo_if_fog', 'meteo_wind_direction_max_inst', 'meteo_amplitude_tn_tx', 'prelev_other_volume_sum', 'piezo_station_altitude', 'piezo_station_investigation_depth', 'meteo_wind_speed_avg_10m', 'prelev_volume_0', 'piezo_station_pe_label', 'meteo_time_tn', 'hydro_station_code', 'prelev_volume_2', 'meteo_temperature_min_50cm', 'meteo_temperature_avg', 'meteo_humidity_duration_above_80%', 'prelev_usage_label_0', 'meteo_radiation', 'piezo_continuity_code', 'prelev_volume_obtention_mode_label_2', 'prelev_volume_obtention_mode_label_0', 'piezo_station_longitude', 'meteo_date', 'meteo_snow_thickness_max', 'hydro_status_code', 'hydro_method_code', 'piezo_measure_nature_code', 'meteo_humidity_max', 'meteo_id', 'meteo_altitude', 

In [36]:
x_trained_without_nan = x_trained_without_nan.drop(columns=["meteo_if_thunderstorm"])

In [ ]:
print(y_original.iloc[:19])

In [ ]:

print(y_train.iloc[:19])

In [ ]:
x_trained_without_nan.head()

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', None)        # No line wrap
pd.set_option('display.max_colwidth', None) # No truncation of columns

nan_percentage_5 = x_trained_without_nan.isna().mean() * 100

# Display the percentage of NaN values per column
print(nan_percentage_5.sort_values(ascending=False))

In [37]:

x_trained_without_nan["insee_%_agri"] = x_trained_without_nan["insee_%_agri"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_trained_without_nan["insee_med_living_level"] = x_trained_without_nan["insee_med_living_level"].replace(
    {'N/A - résultat non disponible': 0}  # Replace with 0 or any value you choose
).astype(float)
x_trained_without_nan["insee_%_ind"] = x_trained_without_nan["insee_%_ind"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_trained_without_nan["insee_%_const"] = x_trained_without_nan["insee_%_const"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)

In [38]:
X_final = encoded_categorical_features(x_trained_without_nan)

In [40]:
X_final.shape

(614663, 86)

In [41]:
X_final = pd.DataFrame(X_final)
X_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614663 entries, 0 to 614662
Data columns (total 86 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       614663 non-null  float64
 1   1       614663 non-null  float64
 2   2       614663 non-null  float64
 3   3       614663 non-null  float64
 4   4       614663 non-null  float64
 5   5       614663 non-null  float64
 6   6       614663 non-null  float64
 7   7       614663 non-null  float64
 8   8       614663 non-null  float64
 9   9       614663 non-null  float64
 10  10      614663 non-null  float64
 11  11      614663 non-null  float64
 12  12      614663 non-null  float64
 13  13      614663 non-null  float64
 14  14      614663 non-null  float64
 15  15      614663 non-null  float64
 16  16      614663 non-null  float64
 17  17      614663 non-null  float64
 18  18      614663 non-null  float64
 19  19      614663 non-null  float64
 20  20      614663 non-null  float64
 21  21      61

In [42]:
X_final.fillna(X_final.mean())

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,-1.728503,-0.264676,0.429546,1.400547,0.439126,1.294739,-0.268619,0.199768,-1.313615,0.134788,...,4.316890,0.009779,-0.112152,1.032931,0.150061,-0.184376,0.601890,-0.614812,0.405116,-2.050682
1,-1.728503,-0.199024,0.418547,1.464602,0.517668,1.151053,-0.176702,0.081369,-1.313615,0.134788,...,0.639816,0.009779,-0.112152,0.556129,-0.598751,-0.159971,0.480544,1.890405,-0.013276,-2.050682
2,-1.728503,-0.200623,0.402050,-1.686903,0.370987,1.101211,-0.237550,0.194783,-1.313615,0.134788,...,0.158851,0.009779,-0.112152,0.081924,-0.638866,0.033633,0.527848,-0.283593,-0.270748,-2.050682
3,-1.728503,-0.204916,0.419922,0.725835,1.818634,1.368392,-0.097138,0.021546,-1.313615,0.134788,...,-0.061767,0.009779,-0.112152,0.081279,-0.538578,-0.127637,3.573851,-0.644923,-0.479944,-2.050682
4,-1.728503,-0.191869,-0.825656,0.657509,0.583334,1.135172,-0.126916,0.020300,-1.313615,0.134788,...,0.256839,0.009779,-0.112152,-0.161473,-0.625494,-0.173311,0.793165,0.505309,-0.109828,-2.050682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614658,2.087990,-0.247842,1.226936,-1.101868,-0.431275,-0.186372,1.280268,-1.368091,1.301459,0.134788,...,-0.140049,-0.629604,0.732444,-0.161486,1.186363,-0.204969,0.725294,-0.789454,1.298222,1.665596
614659,2.087990,-0.241530,0.652265,-1.422142,-0.225263,-0.116771,1.256446,-1.364352,1.301459,0.134788,...,-0.140552,-0.629604,0.732444,-0.151699,-0.565322,-0.152533,0.616287,0.041603,-0.665002,1.665596
614660,2.087990,-0.222591,0.943725,-1.187274,-0.328269,-0.118129,1.228014,-1.285835,1.301459,0.134788,...,-0.140049,-0.629604,0.732444,-0.140366,-0.531893,-0.205799,1.854433,-0.500391,-0.512128,1.665596
614661,2.087990,2.302515,0.630268,-0.926784,-0.632137,-0.003567,1.238191,-1.284589,1.301459,0.134788,...,-0.139846,-0.629604,0.732444,-0.146186,-0.625494,0.149231,1.292948,-0.494369,-0.359254,1.665596


In [ ]:
import seaborn as sns

# Correlation heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(X_final.corr(), annot=False, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

__________________________

ML MODEL

In [43]:
dataset = tf.data.Dataset.from_tensor_slices((X_final,y_train ))

In [44]:
# Total number of samples in your dataset
dataset_size = len(X_final)
train_size = int(0.8 * dataset_size)  # 60% for training
val_size = int(0.2 * dataset_size)    # 20% for validation
 # Remaining 20% for testing

# Shuffle the dataset
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset into train, validation, and test sets
train_dataset = dataset.take(train_size)    # First 80%
val_dataset = dataset.skip(train_size).take(val_size)  # Next 20%


# Batch the datasets
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [50]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(86,)),
    Dense(128, activation='relu'),   # First hidden layer
    Dense(64, activation='relu'),                      # Second hidden layer
    Dense(5, activation='softmax')                     # Output layer for 5 classes
])

In [46]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        11,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,845 (77.52 KB)

 Trainable params: 19,845 (77.52 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-10         # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/ML_model_3.keras',             # File to save the best model
    monitor='val_accuracy',       # Metric to monitor for improvement
    mode='max',                   # Mode 'max' for accuracy (since higher is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)

history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=120,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/120


15367/15367 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4083 - loss: 1.3253
Epoch 1: val_accuracy improved from -inf to 0.61362, saving model to ../models/ML_model_2.keras
15367/15367 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.4083 - loss: 1.3253 - val_accuracy: 0.6136 - val_loss: 0.9225 - learning_rate: 5.0000e-04
Epoch 2/120
15367/15367 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6313 - loss: 0.8764
Epoch 2: val_accuracy improved from 0.61362 to 0.69207, saving model to ../models/ML_model_2.keras
15367/15367 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.6313 - loss: 0.8764 - val_accuracy: 0.6921 - val_loss: 0.7393 - learning_rate: 5.0000e-04
Epoch 3/120
15367/15367 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6996 - loss: 0.7259
Epoch 3: val_accuracy improved from 0.69207 to 0.72829, saving model to ../models/ML_model_2.keras
15367/15367 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - accuracy: 0.6996 - loss: 0.7259 - val_accuracy: 0.7283 - val_loss: 0.6552 - learning_rate: 5.0000

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.grid(True)

In [ ]:
from sklearn.metrics import f1_score

# 1. Make predictions on the validation dataset
y_pred = model.predict(val_dataset)

# 2. Convert predictions to class labels if necessary
# If the model's last layer is 'softmax', use np.argmax to get class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# 3. Get the true labels from the validation dataset
# Note: This assumes val_dataset is a tf.data.Dataset object containing (features, labels)
y_true = np.concatenate([y for _, y in val_dataset], axis=0)

# 4. Calculate the F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')  # 'weighted' is typically used for imbalanced classes

# Print the F1 score
print(f'F1 Score: {f1}')

_________________________

XGBOOST MODEL

In [ ]:



mon_X_train, mon_X_test, mon_y_train, mon_y_test = train_test_split(X_final,y_train, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(mon_X_train)
X_test_scaled = scaler.transform(mon_X_test)

# Initialiser le modèle XGBoost
model = XGBClassifier(
    objective="multi:softmax",
    num_class=5,
    random_state=42,
    eval_metric="mlogloss",
    colsample_bytree=0.7,
    learning_rate=0.2,
    max_depth=7,
    n_estimators=500,
    reg_alpha=0.5,
    reg_lambda=0.0,
    subsample=1.0
)

# Entraîner le modèle
model.fit(X_train_scaled, mon_y_train)

# Prédire sur le jeu de test
y_pred = model.predict(X_test_scaled)

# Calculer le F1-score pondéré
f1 = f1_score(mon_y_test, y_pred, average='weighted')
print(f"F1-Score pondéré sur le jeu de test : {f1:.2f}")

In [ ]:
model.save_model("xgboost_model_2.json")

________________________

RANDOM FOREST